In [11]:
import numpy as np
from neural_networks import Model
from neural_networks.preprocessing import one_hot_encoded

In [12]:
model = Model()

model.add_dense_layer(4, 3, activation='relu')
model.add_dense_layer(1, activation='sigmoid')

In [13]:
X_train = np.array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1], [0, 0, 0]])
y_train = np.array([0, 1, 1, 1, 1, 0, 0])

In [14]:
model.set_parameters(lr=0.1, loss='mse', optimization='sgd')

In [ ]:
model.train(X_train, y_train, epochs=1000, batch_size=1)